In [1]:
import numpy as np

def generate_state(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd):
    """
    生成一个系统的随机状态，该系统负责将任务分配给员工。
    
    参数:
    I_citys (int): 城市的数量。
    L_levels (int): 等级的数量。
    W_workdays (int): 员工距离放假的最大工作天数, 一般为7。
    M_servers (int): 员工的数量。
    x_max_task_num (int): 每个城市的最大任务数(本函数中未直接使用)。
    H_home_of_server (list[int]): 代表每个员工家所在城市的列表。
    lambd (np.array): 一个二维数组 (I_citys x_max_task_num L_levels)，代表每个城市和等级的任务到达率。
    
    返回:
    tuple: 包含任务分布矩阵和员工状态列表的元组。

    城市中的任务矩阵，员工状态列表[(员工所在地，距离放假时间)]
    """

    # 生成任务分布矩阵 (n_il)，其维度为 I_citys x_max_task_num L_levels  
    n_il = np.zeros((I_citys, L_levels), dtype=int)  # 用零初始化矩阵
    for i_city in range(I_citys):  # 遍历城市
        for l_level in range(L_levels):  # 遍历等级
            # 为每个城市和等级分配一个基于泊松分布的随机数, 表示随机状态生成的任务数量？#TODO
            n_il[i_city, l_level] = np.random.poisson(lambd[i_city, l_level])
    S0_tasks = n_il  # 任务分布矩阵 (I_citys x_max_task_num L_levels), [0, +∞)
    
    # 初始化一个列表来保存每个员工的状态，给
    S1_servers = []
    for m_server in range(M_servers):  # 遍历所有员工

        # 为员工 'm_server' 随机选择距离放假的工作日数，取值范围 [0, W_workdays]
        w_m = np.random.randint(0, W_workdays + 1)

        # 根据距离放假天数w_m，得到位置i_m，如果距离放假0，则在家里，否则随机一个位置。
        # i_m [1, I_citys]
        if w_m == W_workdays:  # 如果距离放假时间为 0 天，即今天放假，则员工所在城市为家所在城市
            i_m = H_home_of_server[m_server]
        else:  # 否则，为员工随机选择一 个非家乡城市工作
            i_m = np.random.randint(1, I_citys + 1)
            
        # 将员工的状态作为元组（城市，距离放假的工作日数）添加到列表中
        S1_servers.append((i_m, w_m))
    
    # 将任务分布矩阵和员工状态列表合并成一个状态元组
    S = (S0_tasks, S1_servers)    # ((I_citys x_max_task_num L_levels), (M_servers x_max_task_num 1))

    return S  # 返回生成的状态

In [2]:
I_citys = 10
L_levels = 5
W_workdays = 6
M_servers = 40
x_max_task_num = 2
H_home_of_server = [1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,]
lambd = np.random.rand(I_citys, L_levels)
state = generate_state(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)
print(state)

(array([[1, 0, 2, 0, 2],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 3, 1, 2],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 2],
       [2, 0, 0, 0, 0],
       [0, 2, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]]), [(1, 6), (6, 0), (6, 1), (9, 1), (5, 6), (5, 4), (10, 5), (7, 0), (7, 2), (4, 0), (8, 4), (5, 2), (10, 1), (1, 4), (5, 3), (4, 0), (7, 6), (1, 1), (7, 0), (7, 3), (3, 0), (2, 6), (4, 5), (2, 5), (2, 0), (2, 3), (7, 0), (7, 3), (9, 3), (5, 1), (1, 6), (10, 1), (1, 4), (8, 0), (10, 4), (6, 3), (10, 4), (8, 0), (5, 1), (3, 1)])


In [3]:
def aggreg_state(S, Z_cluster_num, X, M_servers, I_citys, L_levels):
    # 函数2
    # 定义一个函数，用于根据给定的参数将复杂的状态 S 压缩成一个简化的状态 barS。
    # 输入:
    #   S: 当前状态，一个复杂的结构，包含两部分信息：
    #      - 一个数组，表示每个城市每个等级的数量(n_il)。
    #      - 一个列表，表示服务员和他们服务的城市及工作日(i_m, w_m)
    #   Z_cluster_num: 一个整数，表示将城市分成多少个聚类。
    #   X: 一个整数，用于计算 N 矩阵中的元素值。
    #   M_servers: 服务员的总数。
    #   I_citys: 城市的总数。
    #   L_levels: 等级的总数。
    # 输出:
    #   barS: 一个元组，表示压缩后的状态，包含以下三个部分：
    #         - N: 一个二维数组，表示每个聚类的等级之和。
    #         - g: 一个数组，表示每个聚类的状态。
    #         - w: 一个整数，表示第一个服务员的工作日数。

    # 计算正在工作的服务员的数量, S[1][m_server][1]即S1_servers[m_server][1]即w_m
    barM = np.sum([1 for m_server in range(M_servers) if S[1][m_server][1] != 0]) # 距离放假时间不等于0
    # 根据城市数量和设定的簇数，将城市分成Z个簇
    cluster = split_list(I_citys, Z_cluster_num)
    # 计算实际的簇数，考虑到可能会有余数
    # num_cluster = divide_reminder(I_citys, Z_cluster_num) # 这个可以替换为下面的
    num_cluster = np.ceil(I_citys / Z_cluster_num).astype(int) # 向上取整
    # 初始化表示各簇状态的数组g
    g = np.zeros(num_cluster)
    
    # 压缩状态的第二部分：计算每个簇的状态
    for z_cluster in range(num_cluster):
        # 统计每个簇中有多少业务员正在工作 
        e_z = np.sum([1 for m_server in range(M_servers) if S[1][m_server][0] in cluster[z_cluster]])
        # 根据工作的业务员数量设置簇的状态
        if e_z == 0:
            g[z_cluster] = 0  # 无业务员工作
        elif e_z <= barM / num_cluster:
            g[z_cluster] = 1  # 工作业务员数量低于或等于平均值
        else:
            g[z_cluster] = 2  # 工作业务员数量高于平均值
    
    # 获取第一个业务员的工作量
    w = S[1][0][1]
    
    # 压缩状态的第一部分：计算每个簇中各等级的数量总和
    N = np.zeros((num_cluster, L_levels))  # 初始化N矩阵
    for z_cluster in range(num_cluster):
        for l in range(L_levels):
            # 对每个簇的每个等级，计算其数量总和，但不超过X
            N[z_cluster][l] = min(X, np.sum([S[0][i-1][l] for i in cluster[z_cluster]]))
            
    # 将计算出的N矩阵、簇的状态数组g和第一个业务员的工作量w组合成新的压缩状态barS
    barS = (N, g, w)        
    return barS

# 将城市列表平均分成Z个聚类
def split_list(I_citys, Z_cluster_num)->list:
    # 创建一个从1到I_citys的城市索引列表
    arr_city_idx = list(range(1, I_citys + 1))
    # 调用函数处理实际的分割
    return split_array_given_array(arr_city_idx, Z_cluster_num)



def split_array_given_array(arr_city_idx, Z_cluster_num)->list:
    """
    将输入数组分割成长度为 Z_cluster_num 的子数组列表。如果数组不能被 Z_cluster_num 整除，
    那么最后一个子数组将包含所有剩余的元素。
    
    参数:
    arr_city_idx (list): 需要被分割的输入数组。
    Z_cluster_num (int): 每个子数组的期望长度。
    
    返回值:
    list: 长度为 Z_cluster_num 的子数组列表，除了可能的最后一个子数组，它包含所有剩余的元素。
    """
    result = []  # 结果列表，用来存储所有的子数组
    quotient = len(arr_city_idx) // Z_cluster_num  # 计算整除的商，即完整子数组的数量
    remainder = len(arr_city_idx) % Z_cluster_num  # 计算余数，即最后一个子数组的元素数量
    
    # 划分可以整除的数组部分
    for i in range(quotient):
        sub_array = arr_city_idx[i * Z_cluster_num:(i + 1) * Z_cluster_num]  # 获取从 i*Z_cluster_num 到 (i+1)*Z_cluster_num 的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    # 如果有余数，则处理剩余部分
    if remainder > 0:
        sub_array = arr_city_idx[-remainder:]  # 获取数组最后余数个元素形成的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    return result  # 返回结果列表

def divide_reminder(num, divisor):
    """
    将一个整数除以另一个整数，并将结果向上取整。
    
    参数:
    num (int): 被除数。
    divisor (int): 除数。
    
    返回值:
    int: 向上取整后的商。
    """
    quotient = num // divisor  # 计算整除的商
    remainder = num % divisor  # 计算余数
    
    # 如果存在余数，则将商向上取整
    if remainder > 0:
        quotient += 1  # 余数大于0，商加一
    
    return quotient  # 返回向上取整后的商

In [4]:
Z_cluster_num=3
X=3
M_servers=40
I_citys=10
L_levels=5
barS=aggreg_state(state, Z_cluster_num, X, M_servers, I_citys, L_levels)
print(barS)

(array([[1., 1., 2., 0., 2.],
       [1., 2., 3., 1., 3.],
       [2., 2., 0., 1., 0.],
       [1., 0., 0., 0., 0.]]), array([2., 2., 2., 1.]), 6)


In [5]:
# print(state)
print(barS)
S = state
# (位置i_m, 放假天数w_m)
mathscr_L = [server_restday[0] for server_restday in state[1]]
print(f"{S=}\n {mathscr_L=}")

(array([[1., 1., 2., 0., 2.],
       [1., 2., 3., 1., 3.],
       [2., 2., 0., 1., 0.],
       [1., 0., 0., 0., 0.]]), array([2., 2., 2., 1.]), 6)
S=(array([[1, 0, 2, 0, 2],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 3, 1, 2],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 2],
       [2, 0, 0, 0, 0],
       [0, 2, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]]), [(1, 6), (6, 0), (6, 1), (9, 1), (5, 6), (5, 4), (10, 5), (7, 0), (7, 2), (4, 0), (8, 4), (5, 2), (10, 1), (1, 4), (5, 3), (4, 0), (7, 6), (1, 1), (7, 0), (7, 3), (3, 0), (2, 6), (4, 5), (2, 5), (2, 0), (2, 3), (7, 0), (7, 3), (9, 3), (5, 1), (1, 6), (10, 1), (1, 4), (8, 0), (10, 4), (6, 3), (10, 4), (8, 0), (5, 1), (3, 1)])
 mathscr_L=[1, 6, 6, 9, 5, 5, 10, 7, 7, 4, 8, 5, 10, 1, 5, 4, 7, 1, 7, 7, 3, 2, 4, 2, 2, 2, 7, 7, 9, 5, 1, 10, 1, 8, 10, 6, 10, 8, 5, 3]


In [6]:
def f_4(S, mathscr_L):
    # 假设 len(mathscr_L) = 7
    # 假设 S 和 mathscr_L 的数据结构如所描述
    # S 是一个二元组 (S0_tasks, S1_servers)，其中 S0_tasks 是任务矩阵
    # mathscr_L 是业务员的等级集合，例如 [l_1, l_2, ..., l_M]
    #S0 (I_citys x_max_task_num L_levels)
    # 首先计算 N_1 和 N_2
    L_levels = 5
    N_1 = [sum(1 for l_m, (i_m, w_m) in zip(mathscr_L, S[1]) if l_m == j and w_m != 0) for j in range(1, L_levels+1)]
    N_2 = [sum(S[0][i][j] for i in range(len(S[0]))) for j in range(L_levels)]
    print(f"{N_1=} {N_2=}")
    # 初始化分类后的等级列表
    mathcal_L = []
    current_class = []
    total_N_1 = 0
    total_N_2 = 0

    for j in range(1, L_levels+1):
        total_N_1 += N_1[j-1]
        total_N_2 += N_2[j-1]
        current_class.append(j)

        if total_N_1 <= total_N_2:
            # 当 N_1 总和小于等于 N_2 总和时，终止当前类的添加
            mathcal_L.append(tuple(current_class))
            current_class = []
            total_N_1 = 0
            total_N_2 = 0

    if current_class:
        # 添加最后一个类
        mathcal_L.append(tuple(current_class))

    return mathcal_L, N_1, N_2


In [7]:
S = state
# (位置i_m, 放假天数w_m)
mathscr_L = [1,1,1,2,3,4,5,2,3,5, 1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,]
print(f"{S=}\n {mathscr_L=}")
mathcal_L, N_1, N_2 = f_4(S, mathscr_L)
print(mathcal_L, N_1, N_2)

S=(array([[1, 0, 2, 0, 2],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 3, 1, 2],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 2],
       [2, 0, 0, 0, 0],
       [0, 2, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]]), [(1, 6), (6, 0), (6, 1), (9, 1), (5, 6), (5, 4), (10, 5), (7, 0), (7, 2), (4, 0), (8, 4), (5, 2), (10, 1), (1, 4), (5, 3), (4, 0), (7, 6), (1, 1), (7, 0), (7, 3), (3, 0), (2, 6), (4, 5), (2, 5), (2, 0), (2, 3), (7, 0), (7, 3), (9, 3), (5, 1), (1, 6), (10, 1), (1, 4), (8, 0), (10, 4), (6, 3), (10, 4), (8, 0), (5, 1), (3, 1)])
 mathscr_L=[1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5]
N_1=[10, 5, 6, 3, 6] N_2=[5, 5, 5, 2, 6]
[(1, 2, 3, 4, 5)] [10, 5, 6, 3, 6] [5, 5, 5, 2, 6]


In [25]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary

def func3(S, L_mathscr, H_home_of_server, r1, c1, c2):
    """
    生成状态 S 到决策 A 的函数，通过解决线性规划问题来最大化收益 R(S, A)。

    参数:
    S (tuple): 当前状态，包含任务矩阵和服务员信息。
    L_mathscr (list): 服务员的等级列表。
    H_home_of_server (list): 服务员的家位置列表。
    r1 (list): 每个等级的收益列表。
    c1 (list of list): I×I 的成本矩阵。
    c2 (float): 常数成本。

    返回:
    list: 最优决策 A，包含每个服务员的位置和等级。
    """

    # 解构状态 S
    n_il, servers_info = S
    M = len(servers_info)  # 服务员数量
    I = len(n_il)          # 城市数量
    L_max = len(r1) - 1    # 最大等级
    print(f"{M=} {I=}")
    # 创建问题
    prob = LpProblem("Maximize_Revenue", LpMaximize)

    # 创建决策变量
    x = {(m, i, l): LpVariable(f"x_{m}_{i}_{l}", cat=LpBinary) for m in range(M) for i in range(1, I + 1) for l in range(L_max + 1)}

    # 添加约束条件
    for m, (i_m, w_m) in enumerate(servers_info):
        if w_m == 0:
            # 约束条件 (1)
            for i in range(1, I + 1):
                for l in range(1, L_max + 1):
                    prob += x[m, i, l] == 0
            prob += x[m, H_home_of_server[m], 0] == 1
        else:
            # 约束条件 (2)
            for i in range(1, I + 1):
                for l in range(1, L_mathscr[m]):
                    prob += x[m, i, l] == 0
    print(f"{I=} {L_max}")
    for i in range(I):
        for l in range(L_max):
            # 约束条件 (3)
            prob += lpSum(x[m, i, l] for m in range(M)) <= n_il[i - 1][l]

    # 目标函数
    prob += lpSum(r1[l] * x[m, i, l] - c1[i_m - 1][i - 1] * x[m, i, l] 
                  for m, (i_m, w_m) in enumerate(servers_info) 
                  for i in range(1, I + 1) 
                  for l in range(L_max + 1)) - c2 * lpSum(n_il[i - 1][l] - lpSum(x[m, i, l] 
                  for m in range(M)) for i in range(1, I + 1) for l in range(L_max + 1))

    # 解决问题
    prob.solve()

    # 构建最优决策 A
    A = [(i, l) for m in range(M) for i in range(1, I + 1) for l in range(L_max + 1) if x[m, i, l].varValue > 0.5]

    return A

In [26]:
import random

In [27]:
S = state
n_il, servers_info = S
L_mathscr = mathscr_L
H_home_of_server = H_home_of_server
r1  = [0, 3500, 3000, 2500, 2000, 1500]
# 假设 I 是城市的数量，这个值应该根据你的具体情况来设置
I = len(n_il)  # 以 n_il 变量中元素的数量来确定城市数量

# 生成随机的成本矩阵 c1
c1 = [[0 if i == j else random.randint(100, 500) for j in range(I)] for i in range(I)]
c2 = 100
func3(S, L_mathscr, H_home_of_server, r1, c1, c2)

M=40 I=10
I=10 5


IndexError: index 5 is out of bounds for axis 0 with size 5

In [26]:
import numpy as np

def f_7(T, x_max_task_num, lambda_il):
    # 生成了每日新到达的任务?
    # T: 表示时间周期，例如天数
    # x_max_task_num: 矩阵元素的最大取值
    # lambda_il: 泊松分布的率参数矩阵 (I_citys x_max_task_num L_levels)

    # 获取 lambda_il 的维度为 I_citys 和 L_levels
    I_citys, L_levels = lambda_il.shape

    # 初始化三维数组
    arriving_tasks_i = np.zeros((T, I_citys, L_levels), dtype=int)
    
    # 生成每个时间步的 I_citys x_max_task_num L_levels 矩阵
    for t in range(T):
        for i in range(I_citys):
            for l in range(L_levels):
                # 使用泊松分布生成矩阵元素
                arriving_tasks_i[t, i, l] = min(np.random.poisson(lambda_il[i, l]), x_max_task_num)
    
    return arriving_tasks_i



In [28]:
# 示例参数
T = 7  # 时间步数量
x_max_task_num = 3  # 最大值
I_citys = 40  # 城市数量
L_levels = 5  # 等级数量
lambda_il = np.random.rand(I_citys, L_levels)  # 生成率参数矩阵

# 生成arriving_tasks_i
arriving_tasks_i = f_7(T, x_max_task_num, lambda_il)
print(arriving_tasks_i.shape, arriving_tasks_i)

(7, 40, 5) [[[3 1 3 2 1]
  [1 0 0 1 1]
  [0 1 2 0 0]
  ...
  [2 0 1 0 0]
  [0 0 0 0 0]
  [1 0 1 0 0]]

 [[0 2 0 3 1]
  [1 0 1 1 0]
  [1 1 1 0 1]
  ...
  [0 0 0 1 0]
  [0 0 0 0 1]
  [0 1 0 1 0]]

 [[0 0 2 0 1]
  [0 2 1 1 1]
  [0 1 0 0 0]
  ...
  [0 0 1 0 0]
  [0 0 0 0 0]
  [0 0 0 1 0]]

 ...

 [[0 1 2 1 2]
  [1 0 0 2 1]
  [1 0 1 0 3]
  ...
  [0 0 0 1 2]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[1 1 1 0 0]
  [2 0 0 2 0]
  [0 0 1 2 3]
  ...
  [0 0 1 0 0]
  [0 1 0 0 1]
  [0 0 1 2 0]]

 [[1 0 0 0 0]
  [0 0 0 0 0]
  [1 1 0 1 0]
  ...
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 2 1]]]
